In [1]:
import pandas as pd
from datetime import datetime
from glob import glob

In [2]:
def findJoinColumn(columns):
    # check for various options used for join columns in data sets
    # return whicever is present
    # pray no more than 1 is used in the same dataset
    colOptions = ['GEOID', 'fips', 'countyFIPS', 'stateFIPS']
    for option in colOptions:
        if option in columns:
            return option

In [3]:
def findDateIndex(columns):
    for column in columns:
        if column.count('-') > 1 or column.count('/') > 1:
            return columns.index(column)

In [4]:
def convertToUsaFacts(columns):
    mapping = {}
    
    for column in columns:
        if '-' not in column:
            mapping[column] = column
        else:
            parts = column.split('-')
            year = parts[0]
            month = parts[1]
            day = parts[2]
            mapping[column] = f"{int(month)}/{int(day)}/{year[2:]}"
    
    return mapping

In [5]:
raw = pd.read_csv("https://raw.githubusercontent.com/COVIDExposureIndices/COVIDExposureIndices/master/dex_data/county_dex.csv")

In [6]:
raw = pd.read_csv("https://raw.githubusercontent.com/COVIDExposureIndices/COVIDExposureIndices/master/dex_data/state_dex.csv")
        

In [7]:
for column in raw.columns:
    print(column)

state
date
dex
num_devices
dex_a
num_devices_a
dex_income_1
num_devices_income_1
dex_income_1_a
num_devices_income_1_a
dex_income_2
num_devices_income_2
dex_income_2_a
num_devices_income_2_a
dex_income_3
num_devices_income_3
dex_income_3_a
num_devices_income_3_a
dex_income_4
num_devices_income_4
dex_income_4_a
num_devices_income_4_a
dex_education_1
num_devices_education_1
dex_education_1_a
num_devices_education_1_a
dex_education_2
num_devices_education_2
dex_education_2_a
num_devices_education_2_a
dex_education_3
num_devices_education_3
dex_education_3_a
num_devices_education_3_a
dex_education_4
num_devices_education_4
dex_education_4_a
num_devices_education_4_a
dex_race_asian
num_devices_race_asian
dex_race_asian_a
num_devices_race_asian_a
dex_race_black
num_devices_race_black
dex_race_black_a
num_devices_race_black_a
dex_race_hispanic
num_devices_race_hispanic
dex_race_hispanic_a
num_devices_race_hispanic_a
dex_race_white
num_devices_race_white
dex_race_white_a
num_devices_race_white

In [8]:
def getDexData(geography):
    # 
    if (geography == "county"):
        raw = pd.read_csv("https://raw.githubusercontent.com/COVIDExposureIndices/COVIDExposureIndices/master/dex_data/county_dex.csv")
        joinCol = 'county'
    else:
        raw = pd.read_csv("https://raw.githubusercontent.com/COVIDExposureIndices/COVIDExposureIndices/master/dex_data/state_dex.csv")
        joinCol = 'GEOID'
        geoidTable = pd.read_csv('./statename_geoid.csv')
        raw = raw.merge(geoidTable, left_on="state", right_on="STUSPS", how="left")
        
    raw['dex'] = raw['dex'].round(1)
    raw['dex_a'] = raw['dex_a'].round(1)

    dex = raw[[joinCol,'date','dex']]
    dex_a = raw[[joinCol,'date','dex_a']]

    # thanks to @piRSquared on stackoverflow for this nifty pivot expressions
    # https://stackoverflow.com/questions/54915215/expressing-time-series-data-in-the-columns-rather-than-the-rows-of-a-dataframe
    dex = dex.pivot_table(index=joinCol, columns='date').swaplevel(0, 1, 1).sort_index(1).reset_index()
    dex.columns = [column[0] for column in list(dex.columns)]

    dex_a = dex_a.pivot_table(index=joinCol, columns='date').swaplevel(0, 1, 1).sort_index(1).reset_index()
    dex_a.columns = [column[0] for column in list(dex_a.columns)]

    return { 'dex': dex, 'dex_a': dex_a}

In [9]:
def parseDex(fileName, dataFrame, dexJoinColumn):
    casesDf = pd.read_csv(fileName)

    dateColumnIndex = findDateIndex(list(casesDf.columns))
    nonDateCols = list(casesDf.columns)[:dateColumnIndex]
    dateCols = list(casesDf.columns)[dateColumnIndex:]
    dateCols.sort()

    joinColumn = findJoinColumn(list(casesDf.columns))
    casesDf = casesDf[nonDateCols]
    
    if 'usafacts' in fileName:
        dataFrame = dataFrame.rename(columns=convertToUsaFacts(list(dataFrame.columns)))
        
    mergedDf = dataFrame.merge(casesDf, left_on=dexJoinColumn, right_on=joinColumn)

    for column in dateCols:
        if column not in mergedDf.columns:
            mergedDf[column] = None
    parsedDf = mergedDf[nonDateCols + dateCols]

    fileName = fileName.split('confirmed_')[1].split('.csv')[0]
    
    return {'fileName': fileName, 'df': parsedDf}

In [13]:
dex_county

,county,2020-01-20,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,...,2021-02-08,2021-02-09,2021-02-10,2021-02-11,2021-02-12,2021-02-13,2021-02-14,2021-02-15,2021-02-16,2021-02-17
0,1001,217.5,119.0,128.8,135.8,231.6,282.2,161.4,118.9,118.2,...,100.3,101.8,115.8,109.3,193.0,402.7,231.7,104.4,71.5,111.6
1,1003,166.3,134.1,127.7,135.0,193.7,206.7,139.1,127.2,121.6,...,95.5,96.5,97.9,115.5,157.6,337.1,187.5,105.0,98.2,109.8
2,1005,85.2,56.2,64.4,75.4,107.6,118.0,64.3,63.1,56.5,...,48.3,43.9,44.9,64.9,81.1,163.0,99.5,62.8,43.7,52.6
3,1007,159.3,88.8,89.3,100.5,156.1,190.3,111.4,81.4,102.9,...,80.5,91.2,85.7,91.1,141.1,346.9,159.7,60.2,14.9,92.6
4,1009,156.4,110.6,127.3,117.6,196.8,207.7,133.0,117.6,113.3,...,130.6,120.5,135.0,122.0,180.1,394.4,257.2,91.5,25.5,119.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,56029,24.5,52.1,32.1,40.1,43.9,51.9,47.3,23.8,35.7,...,12.2,17.6,19.3,15.2,18.6,39.7,22.1,23.4,22.3,19.8
2014,56033,42.9,30.3,35.1,41.4,49.3,37.6,51.2,25.0,24.4,...,18.2,18.8,22.3,24.2,42.8,55.2,31.4,22.1,18.4,18.0
2015,56037,67.7,63.0,53.6,64.9,78.7,76.3,59.2,46.9,54.6,...,44.9,38.9,42.1,47.0,48.7,64.0,59.3,40.1,40.8,42.3
2016,56039,95.9,231.6,177.5,217.5,150.2,131.9,183.7,169.5,109.0,...,123.0,22.1,67.9,131.9,218.8,322.6,109.5,59.8,38.7,41.8


In [14]:
dex_county.to_csv('./dex_county.csv')

In [12]:
county = getDexData("county")

dex_county = county['dex']
dex_a_county = county['dex_a']

state = getDexData("state")

dex_state = state['dex']
dex_a_state = state['dex_a']

fileList = glob('../../public/csv/*confirmed*.csv')

for file in fileList:
    if 'state' in file:
        dexJoinColumn = 'GEOID'
        dataFrame1 = dex_state
        dataFrame2 = dex_a_state
    else:
        dexJoinColumn = 'county'
        dataFrame1 = dex_county
        dataFrame2 = dex_a_county
        
    parsed1 = parseDex(file, dataFrame1, dexJoinColumn)
    parsed2 = parseDex(file, dataFrame2, dexJoinColumn)
    
#     parsed1['df'].to_csv(f'../../docs/csv/covid_county_dex_{parsed1["fileName"]}.csv',index=False)
#     parsed2['df'].to_csv(f'../../docs/csv/covid_county_dex_a_{parsed2["fileName"]}.csv',index=False)
    parsed1['df'].to_csv(f'./csv/covid_county_dex_{parsed1["fileName"]}.csv',index=False)
    parsed2['df'].to_csv(f'./csv/covid_county_dex_a_{parsed2["fileName"]}.csv',index=False)

TypeError: object of type 'NoneType' has no len()

In [ ]:
fileList

In [10]:
dex_county

NameError: name 'dex_county' is not defined